### Implementing Basic Data Validation with Great Expectations
**Description**: Set up a simple data validation using Great Expectations to check the completeness of a dataset.

**Steps**:
1. Installation
2. Initialize Great Expectations
3. Create a Data Context in Python
4. Create an Expectation Suite
5. Load Sample Data and Validate Completeness
6. Run Validations

In [1]:
# write your code from here

import great_expectations as ge
from great_expectations.data_context import BaseDataContext
from great_expectations.core.batch import BatchRequest
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.checkpoint import SimpleCheckpoint
from great_expectations.data_context.types.base import (DataContextConfig, DatasourceConfig, FilesystemStoreBackendDefaults)
import pandas as pd
import os
import shutil

# -------------------------------
# STEP 1: Create Sample Data
# -------------------------------
def generate_health_data():
    data = {
        "patient_id": [1, 2, 3, 4, 5],
        "age": [35, 42, None, 29, 55],
        "diagnosis": ["Hypertension", "Diabetes", "None", None, "Diabetes"]
    }
    return pd.DataFrame(data)

# -------------------------------
# STEP 2: Initialize GE Context
# -------------------------------
def setup_ge_context(project_root="ge_basic_project"):
    if os.path.exists(project_root):
        shutil.rmtree(project_root)

    context_config = DataContextConfig(
        datasources={},
        store_backend_defaults=FilesystemStoreBackendDefaults(root_directory=project_root),
        expectations_store_name="expectations_store",
        validations_store_name="validations_store",
        evaluation_parameter_store_name="evaluation_parameter_store",
    )

    context = BaseDataContext(project_config=context_config)
    print(f"[Info] Great Expectations context initialized at: {project_root}")
    return context

# -------------------------------
# STEP 3: Register Data Source
# -------------------------------
def register_pandas_datasource(context):
    datasource_config = {
        "name": "my_pandas_source",
        "class_name": "Datasource",
        "execution_engine": {"class_name": "PandasExecutionEngine"},
        "data_connectors": {
            "runtime_data_connector": {
                "class_name": "RuntimeDataConnector",
                "batch_identifiers": ["default_identifier_name"],
            }
        },
    }
    context.add_datasource(**datasource_config)
    print("[Info] Pandas data source registered.")

# -------------------------------
# STEP 4: Create Expectation Suite
# -------------------------------
def create_expectation_suite(context, suite_name="healthcare_suite"):
    try:
        suite = context.create_expectation_suite(expectation_suite_name=suite_name, overwrite_existing=True)
        print(f"[Info] Created Expectation Suite: {suite_name}")
    except Exception as e:
        print(f"[Error] Could not create suite: {e}")
    return suite_name

# -------------------------------
# STEP 5: Add Expectations
# -------------------------------
def add_expectations(context, suite_name, df):
    validator = context.get_validator(
        batch_request={
            "datasource_name": "my_pandas_source",
            "data_connector_name": "runtime_data_connector",
            "data_asset_name": "health_asset",
            "runtime_parameters": {"batch_data": df},
            "batch_identifiers": {"default_identifier_name": "default"},
        },
        expectation_suite_name=suite_name,
    )

    # Add completeness expectations
    validator.expect_column_to_exist("patient_id")
    validator.expect_column_values_to_not_be_null("patient_id")
    validator.expect_column_values_to_not_be_null("age")
    validator.expect_column_values_to_not_be_null("diagnosis")

    validator.save_expectation_suite(discard_failed_expectations=False)
    print("[Info] Expectations added and saved.")

# -------------------------------
# STEP 6: Run Validation
# -------------------------------
def run_validation(context, suite_name, df):
    checkpoint = SimpleCheckpoint(
        name="basic_validation_checkpoint",
        data_context=context,
        validations=[
            {
                "batch_request": {
                    "datasource_name": "my_pandas_source",
                    "data_connector_name": "runtime_data_connector",
                    "data_asset_name": "health_asset",
                    "runtime_parameters": {"batch_data": df},
                    "batch_identifiers": {"default_identifier_name": "default"},
                },
                "expectation_suite_name": suite_name,
            }
        ],
    )

    results = checkpoint.run()
    success = results["success"]
    print(f"[Validation Result] ✅ Passed: {success}")
    return results

# -------------------------------
# MAIN EXECUTION
# -------------------------------
if __name__ == "__main__":
    print("=== Great Expectations: Basic Data Completeness Check ===\n")

    # Step 1: Simulate data
    df = generate_health_data()

    # Step 2: Setup context
    context = setup_ge_context()

    # Step 3: Register Pandas datasource
    register_pandas_datasource(context)

    # Step 4: Create Expectation Suite
    suite_name = create_expectation_suite(context)

    # Step 5: Add expectations
    add_expectations(context, suite_name, df)

    # Step 6: Run validations
    results = run_validation(context, suite_name, df)

    print("\n=== DONE ===")

ModuleNotFoundError: No module named 'great_expectations'